In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval()

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [2]:
def clear_gpu():
    import gc, torch
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

clear_gpu()


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

# Classification

In [4]:
import json
file_path = "/kaggle/input/classification/classification_instruction_data_en.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    data = [json.loads(line.strip()) for line in f]

In [5]:
valid_labels = {"negative", "positive", "neutral"}
data = [sample for sample in data if sample['output'].strip().lower() in valid_labels]

In [6]:
import re

def extract_answer(text):
    matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
    if matches:
        return matches[-1]  
    return None

In [7]:
def extract_target(text):
  for line in text.splitlines()[::-1]:
    if line.strip().startswith("####"):
      return line.strip().replace("####","").strip()
  return None

In [8]:
def split_cot_and_answer(output_text):
    lines = output_text.strip().splitlines()
    cot_lines = [line for line in lines if not line.strip().startswith("####")]
    answer = extract_target(output_text)
    return "\n".join(cot_lines), answer

**Zero-shot**

In [9]:
true_label = []
for i in range(200):
    true_label.append(data[i]['output'])

In [10]:
acc = 0
true_results = []
for i in range(200):
  zero_shot_prompt = "Classify the sentiment of this sentence as positive, negative, or neutral:\n{text}"
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(data[i]['output'])
  print(de)
  if de == data[i]['output']:
    acc = acc+1
    true_results.append(de)
print(acc/200)

neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
negative
neutral
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neu

In [11]:
from collections import Counter

counts = Counter(true_results)

print(counts)


Counter({'neutral': 157})


In [12]:
counts = Counter(true_label)

print(counts)

Counter({'neutral': 157, 'positive': 38, 'negative': 5})


Zero-shot + CoT (ReAct)

In [16]:
acc = 0
true_results = []
results = []
for i in range(200):
  zero_shot_prompt = f"""
  You are a classifier assistant.Classify the sentiment of this sentence as positive, negative, or neutral:\n{data[i]['input']}. \n
  A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de == data[i]['output']:
    acc = acc+1
    true_results.append(de)
  results.append(de)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

0.805
Counter({'neutral': 152, 'positive': 6, 'negative': 3})
Counter({'neutral': 186, 'positive': 7, 'negative': 7})


In [17]:
acc = 0
true_results = []
results = []
for i in range(200):
  zero_shot_prompt = f"""
  You are a classifier assistant.Classify the sentiment of this sentence as positive, negative, or neutral:\n{data[i]['input']}.
  Recorrect with other LLM models. \n
  A: Let's think step by step. The sentiment is """
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de == data[i]['output']:
    acc = acc+1
    true_results.append(de)
  results.append(de)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

0.77
Counter({'neutral': 154})
Counter({'neutral': 197, 'negative': 3})


Zero-shot + ToT

In [18]:
import re

def extract_labels_from_thoughts(text):
    matches = re.findall(r'\b(positive|negative|neutral)\b', text.lower())
    return list(set(matches))  # loại trùng nếu cần


In [19]:
import re

def clean_label(text):
    match = re.search(r"\b(positive|negative|neutral)\b", text.lower())
    return match.group(1) if match else "UNKNOWN"


In [ ]:
def generate_strategy(prompt_base, strategy_index):
    prompt = prompt_base + f"Strategy {strategy_index + 1}:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
# acc = 0
# for i in range(200):
#     prompt_base = zero_shot_ToT(data[i])
#     strategies = [generate_strategy(prompt_base, j) for j in range(3)]
#     labels = extract_labels_from_thoughts("\n".join(strategies))
#     if labels:
#         prompt = followup_prompt(", ".join(labels))
#         input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
#         output = model.generate(
#             **input_ids,
#             max_length=20,
#             num_beams=5,
#             no_repeat_ngram_size=2,
#             early_stopping=True
#         )
#         final_label = tokenizer.decode(output[0], skip_special_tokens=True)
#         final_label = clean_label(final_label)
#         if final_label == data[i]['output']:
#             acc = acc+1
# print(acc/200)

In [ ]:
# acc = 0
# for i in range(200):
#     prompt_base = zero_shot_ToT(data[i])
#     strategies = [generate_strategy(prompt_base, j) for j in range(5)]
#     labels = extract_labels_from_thoughts("\n".join(strategies))
#     if labels:
#         prompt = followup_prompt(", ".join(labels))
#         input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
#         output = model.generate(
#             **input_ids,
#             max_length=20,
#             num_beams=5,
#             no_repeat_ngram_size=2,
#             early_stopping=True
#         )
#         final_label = tokenizer.decode(output[0], skip_special_tokens=True)
#         final_label = clean_label(final_label)
#         if final_label == data[i]['output']:
#             acc = acc+1
# print(acc/200)

In [ ]:
# acc = 0
# for i in range(200):
#     prompt_base = zero_shot_ToT(data[i])
#     strategies = [generate_strategy(prompt_base, j) for j in range(10)]
#     labels = extract_labels_from_thoughts("\n".join(strategies))
#     if labels:
#         prompt = followup_prompt(", ".join(labels))
#         input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
#         output = model.generate(
#             **input_ids,
#             max_length=20,
#             num_beams=5,
#             no_repeat_ngram_size=2,
#             early_stopping=True
#         )
#         final_label = tokenizer.decode(output[0], skip_special_tokens=True)
#         final_label = clean_label(final_label)
#         if final_label == data[i]['output']:
#             acc = acc+1
# print(acc/200)

Zero-shot + CoT + SC

In [20]:
from collections import Counter
def self_consistency(prompt, num_samples=5, max_len=50):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=max_len,
            do_sample=True,           
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    counts = Counter(outputs)
    return counts.most_common(1)[0][0], counts

In [43]:
def zero_shot_CoT(text):
  return f"""
  You are a classifier assistant.
  Q: {text['input'] } A: Let's think step by step. Classify the sentiment of this sentence as positive, negative, or neutral:\n{data[i]['input']}. The sentiment is """

In [44]:
prompt = zero_shot_CoT(data[0])
print(prompt)
samples = self_consistency(prompt, num_samples=5)
best_answer, all_votes = majority_vote(samples)

print("Self-Consistent Answer:", best_answer)
print("All votes:", all_votes)


  You are a classifier assistant.
  Q: between 1900 and 1920 where did most of the migrants to the united states come from A: Let's think step by step. Classify the sentiment of this sentence as positive, negative, or neutral:
Write a funny, clever happy birthday message for my beautiful, smart, sassy best friend to post on my Instagram story. The sentiment is 
Self-Consistent Answer: positive
All votes: Counter({'positive': 5})


In [45]:
def zero_shot_CoT_SC(text, num_samples=5, max_len=50, do_print=False):
        prompt = zero_shot_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('Self-consistent answer: ', best_answer)
            print('All votes: ', all_votes)
        return best_answer, all_votes

In [48]:
acc = 0
true_results = []
results = []
for i in range(200):
    best_answer, all_votes = zero_shot_CoT_SC(data[i])
    if best_answer.lower() == data[i]['output']:
        acc = acc + 1
        true_results.append(best_answer)
    results.append(best_answer)

In [49]:
print(acc/200)

0.685


In [50]:
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

Counter({'neutral': 118, 'positive': 15, 'negative': 4})
Counter({'neutral': 140, 'positive': 46, 'negative': 12, 'positive. The answer: positive.': 1, 'positive because the test is a positive thing. The answer: positive.': 1})


In [51]:
acc = 0
true_results = []
results = []
for i in range(200):
    best_answer, all_votes = zero_shot_CoT_SC(data[i],3)
    if best_answer.lower() == data[i]['output']:
        acc = acc + 1
        true_results.append(best_answer.lower())
    results.append(best_answer)
print('Accuracy', acc/200)
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

Accuracy 0.635
Counter({'neutral': 108, 'positive': 14, 'negative': 5})
Counter({'neutral': 131, 'positive': 54, 'negative': 13, 'positive. The answer: positive.': 1, 'neutral because you cannot be positive and negative at the same time. The answer: neutral.': 1})


In [89]:
def select_best_path(thoughts, text, do_print=False):
    prompt = f"""
You are an expert in emotional reasoning. Read the sentence below and analyze the following reasoning paths carefully.

Context: "{text}"

Reasoning paths:
{chr(10).join([
    f"{i+1}. {t}" for i, t in enumerate(thoughts)
])}


Steps:
1. Carefully review the reasoning paths above. Think about emotional polarity, tone, contrast, and subjectivity.
2. Choose the most thoughtful reasoning path (not just a label like 'positive').
3. Output in this exact JSON format:
{{
  "final_sentiment": "positive | negative | neutral",
  "reasoning": "Your explanation using the best reasoning path"
}}

Do not output only the sentiment. Always include explanation inside the JSON block.
"""

    if do_print:
        print('Prompt:\n', prompt)

    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
        max_length=350,
        do_sample=False
    )
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

In [77]:
    def expand_thoughts( prompt, n=3, max_len=50):
        input = tokenizer(prompt, return_tensors='pt').to('cuda')
        outputs = model.generate(
            **input,
            max_length = max_len,
            num_return_sequences=n,
            do_sample = True,
            top_p=0.95,
            temperature=0.7
        )
        thoughts = [tokenizer.decode(output, skip_special_tokens=True).strip() for output in outputs]
        return thoughts

In [94]:
def recursive_expand_tree(prompt, depth, breadth, do_print=False):
    if depth == 0:
        return [prompt]

    expanded = expand_thoughts(prompt, n=breadth, max_len=100)
    if do_print:
        print(f"[Depth {depth}] Expand prompt:\n{prompt}")
        print(f"[Depth {depth}] Got thoughts:\n", expanded)

    tree = []
    for thought in expanded:
        sub_tree = recursive_expand_tree(thought, depth - 1, breadth, do_print)
        tree.extend(sub_tree)

    return tree

Zero-shot ToT (basic)

In [41]:
def zero_shot_ToT( text):
        return f"""
        You are a reasoning classifier assistant. Consider multiple possibilities, classify label for each path (negative, positive, neutral)
         and choose the most reasonable one.
        Q: {text}
        Option 1: Analyze based on emotional keywords.
        Option 2: Consider the context and tone.
        Option 3: Check for contrast or negation.
        Evaluate each option and choose the best.
        Final Answer (label: positive, negative,or neutral ): ?
        """

In [42]:
acc = 0
true_results = []
results = []
for i in range(200):
    prompt = zero_shot_ToT(data[i]['input'])
    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
        max_length=20,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    de = tokenizer.decode(output[0], skip_special_tokens=True)
    if de.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(de)
    results.append(de.lower())
print('Accuracy', acc/200)
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

Accuracy 0.825
Counter({'neutral': 154, 'positive': 7, 'negative': 4})
Counter({'neutral': 186, 'positive': 9, 'negative': 5})


In [52]:
acc = 0
true_results = []
results = []
for i in range(200):
    prompt = zero_shot_ToT(data[i]['input'])
    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
        max_length=200,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    de = tokenizer.decode(output[0], skip_special_tokens=True)
    if de.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(de)
    results.append(de.lower())
print('Accuracy', acc/200)
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

Accuracy 0.825
Counter({'neutral': 154, 'positive': 7, 'negative': 4})
Counter({'neutral': 186, 'positive': 9, 'negative': 5})


In [54]:
from datasets import load_dataset

dataset = load_dataset("allenai/social_i_qa")

# Hiển thị một mẫu từ tập huấn luyện
print(dataset["train"][0])


The repository for allenai/social_i_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/social_i_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/33410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1954 [00:00<?, ? examples/s]

{'context': 'Cameron decided to have a barbecue and gathered her friends together.', 'question': 'How would Others feel as a result?', 'answerA': 'like attending', 'answerB': 'like staying home', 'answerC': 'a good friend to have', 'label': '1'}


In [57]:
dataset['train']

Dataset({
    features: ['context', 'question', 'answerA', 'answerB', 'answerC', 'label'],
    num_rows: 33410
})

In [ ]:
def convert_to_instruction_format(example):
    try:
        choices = [example["answerA"], example["answerB"], example["answerC"]]
        label = int(example["label"])
        if label not in [0, 1, 2]:
            return None  # bỏ qua mẫu lỗi
        correct_answer = choices[label]
        return {
            "instruction": "Answer the question (Inference)",
            "input": f"{example['context']} {example['question']} Choices: A) {choices[0]}, B) {choices[1]}, C) {choices[2]}",
            "output": f"Answer: {['A', 'B', 'C'][label]}\nExplanation: {correct_answer}"
        }
    except (KeyError, ValueError, IndexError):
        return None  # bỏ qua nếu có lỗi

# Chuyển đổi toàn bộ tập dữ liệu
converted_data = [convert_to_instruction_format(ex) for ex in dataset['train']]

# Ghi vào tệp JSON
with open('/kaggle/working/reasoning_social.json', 'w', encoding='utf-8') as f:
    json.dump(converted_data, f, ensure_ascii=False, indent=2)

In [91]:
def wrap_reasoning_to_json(reasoning_text, do_print=False):
    prompt = f"""
Given the reasoning below, output the result in this exact JSON format:
{{
  "final_sentiment": "positive | negative | neutral",
  "reasoning": "your explanation..."
}}

Reasoning:
{reasoning_text}
"""
    if do_print:
        print("[Wrap Prompt]\n", prompt)

    input_ids = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(
        **input_ids,
        max_length=300,
        do_sample=False
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()


In [92]:
def zero_shot_step_by_step_sentiment(text, do_print=False):
    step_by_step_prompt = f"""
You are a reasoning assistant. Let's analyze the sentiment of the following sentence **step-by-step**.

Sentence: "{text}"

Step 1: Is this sentence factual or emotional?
Step 2: If emotional, describe the emotional tone. If factual, say why it has no emotional tone.
Step 3: Based on the tone and content, what is the most accurate sentiment? (positive, negative, neutral)

Give reasoning clearly and concisely.
"""
    if do_print:
        print("[Prompt]\n", step_by_step_prompt)

    input_ids = tokenizer(step_by_step_prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(
        **input_ids,
        max_length=300,
        do_sample=False
    )
    reasoning = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Wrap into final JSON format
    return wrap_reasoning_to_json(reasoning, do_print=do_print)


Zero-shot ToT BFS

In [95]:
result = zero_shot_ToT_expanded(data[0]['input'], breadth = 3, do_print=True)
print(result)

You are a sentiment classifier assistant. Use multiple reasoning paths to determine the sentiment of the sentence: "between 1900 and 1920 where did most of the migrants to the united states come from". Consider emotional tone, polarity, and contrast. Answer by exploring multiple reasoning paths clearly.
[Depth 2] Expand prompt:
You are a sentiment classifier assistant. Use multiple reasoning paths to determine the sentiment of the sentence: "between 1900 and 1920 where did most of the migrants to the united states come from". Consider emotional tone, polarity, and contrast. Answer by exploring multiple reasoning paths clearly.
[Depth 2] Got thoughts:
 ['positive', 'negative', 'positive']
[Depth 1] Expand prompt:
positive
[Depth 1] Got thoughts:
 ['i have to say that i do not want to give up the fml', "i'm just so thankful that i can go back to school and see my baby boy so soon!!", 'the new album is really good']
[Depth 1] Expand prompt:
negative
[Depth 1] Got thoughts:
 ["I am a fan o

**Few-shots**

In [96]:
def few_shots_direct(text):
        return f"""Classify the sentiment of each sentence as positive, neutral, or negative.
                Sentence: I love this! → positive
                Sentence: This is terrible. → negative
                Sentence: It works as expected. → neutral
                Sentence: {text} →"""


In [97]:
acc = 0
results = []
true_results = []
for i in range(200):
  prompt = few_shots_direct(data[i]['input'])
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de.lower() == data[i]['output']:
    acc = acc+1
    true_results.append(de.lower())
  results.append(de.lower())
print(acc/200)
counts = Counter(results)
print(counts)
counts = Counter(true_results)
print(counts)

0.775
Counter({'neutral': 170, 'positive': 20, 'negative': 10})
Counter({'neutral': 141, 'positive': 11, 'negative': 3})


Few-shot + CoT (ReAct)

In [99]:
acc = 0
results = []
true_results = []
for i in range(200):
  few_shots_CoT =  f"""
    Classify the sentiment of each sentence as positive, neutral, or negative. Let's explain step by step.
    Sentence: I love this! → Reason: Strong positive words → Answer: positive
    Sentence: This is terrible. → Reason: Strong negative tone → Answer: negative
    Sentence: It works as expected. → Reason: Neutral tone and wording → Answer: neutral
    Sentence: {data[i]['input']} → Reason: Let's think step by step. The sentiment is """
  inputs = tokenizer(few_shots_CoT, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de.lower() == data[i]['output']:
    acc = acc+1
    true_results.append(de.lower())
  results.append(de.lower())
print(acc/200)
counts = Counter(results)
print(counts)
counts = Counter(true_results)
print(counts)

0.0
Counter({'answer: neutral': 164, 'answer: positive': 24, 'answer: negative': 12})
Counter()


In [100]:
acc = 0
results = []
true_results = []
for i in range(200):
  few_shots_prompt = f"""
  Classify the sentiment of each sentence as positive, neutral, or negative. Let's explain step by step.
                Sentence: I love this! → Reason: Strong positive words → Answer: positive
                Sentence: This is terrible. → Reason: Strong negative tone → Answer: negative
                Sentence: It works as expected. → Reason: Neutral tone and wording → Answer: neutral
                Sentence: {data[i]['input']} → Reason:"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=7,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  results.append(de)
  if de == data[i]['output']:
    acc = acc+1
    true_results.append(de)
print(acc/200)
from collections import Counter
counts = Counter(results)
print('Number of results',counts)
counts = Counter(true_results)
print('Number of true results', counts)

0.015
Number of results Counter({'Answer: neutral': 164, 'Answer: positive': 23, 'Answer: negative': 10, 'neutral': 3})
Number of true results Counter({'neutral': 3})


*thử đảo vị trí các ví dụ*

In [101]:
acc = 0
results = []
true_results = []
for i in range(3, 203):
  few_shots_prompt = f"""Classify the sentiment of each sentence as positive, neutral, or negative. Let's explain step by step.
                Sentence: This is terrible. → Reason: Strong negative tone → Answer: negative
                Sentence: It works as expected. → Reason: Neutral tone and wording → Answer: neutral
                Sentence: I love this! → Reason: Strong positive words → Answer: positive         
                Sentence: {data[i]['input']} → Reason:"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de.lower() == data[i]['output']:
    acc = acc+1
    true_results.append(de.lower())
  results.append(de.lower())

In [102]:
print(acc/200)
print(Counter(true_results))
counts = Counter(results)
print(counts)

0.035
Counter({'neutral': 7})
Counter({'answer: neutral': 160, 'answer: positive': 23, 'answer: negative': 10, 'neutral': 7})


In [103]:
  def few_shots_CoT(text):
    return f"""
You're a sentiment classifier assistant. Classify the sentiment of the sentence into one of the following labels: **negative**, **positive**, or **neutral**. Use chain-of-thought reasoning to explain your answer clearly. Re-evaluate your classification using reasoning, as if double-checking with another LLM.

Examples:

Q: Although I had high hopes, this product was a huge disappointment.  
Let's think step by step.  
1. The person had high hopes.  
2. But the product was a disappointment.  
3. The overall sentiment is negative.  
A: negative

Q: How can I print mouse coordinates in Rust GTK on a drawing area when clicked?  
Let's think step by step.  
1. This is a factual and technical question.  
2. It expresses no emotion.  
3. Therefore, the sentiment is neutral.  
A: neutral

Q: I hate this chat  
Let's think step by step.  
1. The word "hate" expresses strong negative emotion.  
2. Therefore, the sentiment is negative.  
A: negative

Q: Efficient and fast  
Let's think step by step.  
1. Both "efficient" and "fast" are positive qualities.  
2. Therefore, the sentiment is positive.  
A: positive

Q: {text}  
Let's think step by step. A: The sentiment is (negative, positive or neutral)?
"""



In [104]:
acc = 0
results = []
true_results = []
for i in range(200):
  inputs = tokenizer(few_shots_CoT(data[i]['input']), return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=7,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  results.append(de.lower())
  if de.lower() == data[i]['output']:
    acc = acc+1
    true_results.append(de.lower())
print('Accuracy ', acc/200)
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

Accuracy  0.575
Counter({'neutral': 112, 'positive': 2, 'negative': 1})
Counter({'neutral': 132, 'a: neutral': 34, 'a: negative': 3, 'a: positive': 3, 'positive': 2, 'neutral. i am working on similarity between lion life and human with a purpose. so, the answer is neutral.': 2, 'neutral. tom threw his school bag down to ray after he reached the top of the stairs. therefore, the sentiment is neutral.': 1, "positive. you are a branding consultant with creative mind, give me 30 naming ideas for a baby's website for parents in table format. therefore, the sentiment is positive. a: positive": 1, '1. how to tell your mother you are visiting a friend for an evening in he style of shakespeare. 2. neutral. 3. therefore, the sentiment is neutral. a: neutral': 1, 'neutral. i am doing a business finance assignment. so, the answer is neutral.': 1, 'neutral. you said you are not a physical device, but i believe you need one or two parts of physical devices like. so, the answer is neutral.': 1, "neut

Few-shots + CoT + SC

In [105]:
def few_shots_CoT(text):
        return f"""Classify the sentiment of each sentence as positive, neutral, or negative. Let's explain step by step.
                Sentence: I love this! → Reason: Strong positive words → Answer: positive
                Sentence: This is terrible. → Reason: Strong negative tone → Answer: negative
                Sentence: It works as expected. → Reason: Neutral tone and wording → Answer: neutral
                Sentence: {text} → Reason:"""
def few_shots_CoT_SC(text, num_samples=5, max_len=50, do_print=False):
        prompt = few_shots_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('Answers: ', samples)
            print('Self-consistent answer: ', best_answer)
            print('All votes: ', all_votes)
        return best_answer, all_votes

In [107]:
acc = 0
results = []
true_results = []
for i in range(200):
    best_answer, all_votes = few_shots_CoT_SC(data[i], do_print=True)
    results.append(best_answer.lower())
    if best_answer.lower() == data[i]['output']:
        acc = acc + 1
        true_results.append(best_answer.lower())
print(acc/200)
print(Counter(true_results))
print(Counter(results))

Answers:  ['Answer: neutral', 'Answer: neutral', 'Answer: neutral', 'Answer: neutral', 'Answer: neutral']
Self-consistent answer:  Answer: neutral
All votes:  Counter({'Answer: neutral': 5})
Answers:  ['Answer: neutral', 'Answer: neutral', 'Answer: neutral', 'Answer: neutral', 'Answer: neutral']
Self-consistent answer:  Answer: neutral
All votes:  Counter({'Answer: neutral': 5})
Answers:  ['Answer: neutral', 'Answer: neutral', 'Answer: neutral', 'Answer: neutral', 'Answer: neutral']
Self-consistent answer:  Answer: neutral
All votes:  Counter({'Answer: neutral': 5})
Answers:  ['Answer: neutral', 'Answer: neutral', 'Answer: neutral', 'Answer: neutral', 'Answer: neutral']
Self-consistent answer:  Answer: neutral
All votes:  Counter({'Answer: neutral': 5})
Answers:  ['neutral', 'neutral', 'neutral', 'neutral', 'neutral']
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 5})
Answers:  ['Answer: positive', 'Answer: positive', 'Answer: positive', 'Answer: positive', 'Answer: n

Few-shots + Self-evaluation ToT 

In [108]:
def few_shots_ToT(text):
        return f"""You are a sentiment analysis assistant. For each sentence, explain your reasoning and then classify it as **positive**, **negative**, or **neutral**.
            
            Example 1:
            Sentence: "The movie was boring but had a good ending."
            Reasoning: Boring → negative, Good ending → positive. The ending left a stronger impression → Sentiment: positive
            
            Example 2:
            Sentence: "I expected more, but I guess it’s fine."
            Reasoning: Disappointment shows unmet expectations, but “it’s fine” shows acceptance. Mixed feelings → Sentiment: neutral
            
            Example 3:
            Sentence: "The product is overpriced and doesn’t work well."
            Reasoning: Overpriced and malfunctioning product indicates frustration → Sentiment: negative
            
            Now analyze the following sentence:
            Sentence: "{text}"
            Reasoning:"""

In [109]:
acc = 0
results = []
true_results = []
for i in range(200):
    prompt = few_shots_ToT(data[i]['input'])
    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
        max_length=200,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    de = tokenizer.decode(output[0], skip_special_tokens=True)
    if de.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(de.lower())
    results.append(de.lower())
print('Accuracy', acc/200)
print(Counter(true_results))
print(Counter(results))

Accuracy 0.815
Counter({'neutral': 157, 'positive': 4, 'negative': 2})
Counter({'neutral': 194, 'positive': 4, 'negative': 2})


In [ ]:
def few_shots_ToT_expanded( text, depth=2, breadth=3, do_print=False):
      root_prompt = f"""You are a sentiment analysis assistant. For each sentence, explain your reasoning and then classify it as **positive**, **negative**, or **neutral**.
            
            Example 1:
            Sentence: "The movie was boring but had a good ending."
            Reasoning: Boring → negative, Good ending → positive. The ending left a stronger impression → Sentiment: positive
            
            Example 2:
            Sentence: "I expected more, but I guess it’s fine."
            Reasoning: Disappointment shows unmet expectations, but “it’s fine” shows acceptance. Mixed feelings → Sentiment: neutral
            
            Example 3:
            Sentence: "The product is overpriced and doesn’t work well."
            Reasoning: Overpriced and malfunctioning product indicates frustration → Sentiment: negative
            
            Now analyze the following sentence:
            Sentence: "{text}"
            Reasoning:"""
        tree = recursive_expand_tree(root_prompt, depth=depth, breadth=breadth, do_print=do_print)
        best_path = select_best_path(tree, text)
        return best_path

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
    result = few_shots_ToT_expanded(data[i]['input'], breadth = 3)
    if result.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(result.lower())
    results.append(result)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
    result = few_shots_ToT_expanded(data[i]['input'], depth = 3, breadth = 1)
    if result.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(result.lower())
    results.append(result)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
    result = few_shots_ToT_expanded(data[i]['input'], depth = 2, breadth = 2)
    if result.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(result.lower())
    results.append(result)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

# QA

(hỏi đáp đơn giản)

In [ ]:
clear_gpu()

**Zero-shot**

In [123]:
import json

with open('/kaggle/input/simple-qa/simple_qa (2).json', 'r', encoding='utf-8') as f:
        qa = json.load(f)

In [124]:
qa[:3]

[{'instruction': 'Answer the question',
  'input': 'between 1900 and 1920 where did most of the migrants to the united states come from',
  'output': 'During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world.'},
 {'instruction': 'Answer the question',
  'input': 'What does inioluwa mean',
  'output': 'Inioluwa is a Nigerian name that means "God\'s gift" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God.'},
 {'instruction': 'Answer the question',
  'input': 'what is CVD',
  'output': 'CVD stands for Cardiovascular Disease. It is a group of conditions that affect the heart and blood vessels. These conditions include coronary artery disease (which can lead

In [125]:
len(qa)

98

In [126]:
def zero_shot_direct(text):
        return f"""
        Answer the following question by your knowledge.
        Q: {text}. A: Answer the question
        """

In [127]:
print(qa[0]['input'])
print(qa[0]['output'])

between 1900 and 1920 where did most of the migrants to the united states come from
During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world.


In [128]:
input = tokenizer(zero_shot_direct(qa[0]['input']), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Mexico


In [129]:
test_error = """Q: In the early 20th century, where did most of the migrants to the united states come from
. A: Answer the question, which's country ? """

input = tokenizer(test_error, return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))


( D).


moi lan mot nuoc ~

In [130]:
print(qa[1]['input'])
print(qa[1]['output'])

What does inioluwa mean
Inioluwa is a Nigerian name that means "God's gift" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God.


In [131]:
input = tokenizer(zero_shot_direct(qa[1]['input']), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

oh how lovely


Zero-shot + CoT

In [132]:
def zero_shot_CoT(text):
        return f"""
                Answer the following question by your knowledge.
                Q: {text}. A: Let's think step by step and explain why. Answer the question
                """

In [ ]:
clear_gpu()

In [ ]:
clear_gpu()

In [133]:
input = tokenizer(zero_shot_CoT(qa[0]['input']), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The important part for answering this question is: Between 1900 and 1920, most of the migrants to the United States came from Europe. So, the final answer is Europe.


EM

In [ ]:
import re

def extract_final_answer(text):
    match = re.search(r'final answer is\s*(.+?)(?:\.|$)', text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    sentences = re.split(r'[.?!]', text)
    if sentences:
        return sentences[-2].strip()
    return text.strip()


In [ ]:
def exact_match(prediction, ground_truth):
    pred_ans = extract_final_answer(prediction)
    gt_ans = extract_final_answer(ground_truth)
    
    def normalize_text(text):
        text = text.lower().strip()
        text = re.sub(r'[^\w\s]', '', text)
        return text
    
    return normalize_text(pred_ans) == normalize_text(gt_ans)

In [140]:
for i in range(10):
    input = tokenizer(zero_shot_CoT(qa[i]['input']), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)

The relevant information is: Between 1900 and 1920, most of the migrants to the United States came from Germany. So, the final answer is Germany.
Inioluwa is a word in the pidgin English language. Inia is the meaning of iniowa, and is the antonym of ine. So the final answer is inia.
To prevent coronary artery disease, people should eat a healthy diet, get regular exercise, and get regular screenings for blood clots. Cardiovascular disease is a disease characterized by high blood pressure, heart attack, stroke, and heart failure. So the final answer is CVD.
The relevant information to answer the above question is: Chicken is a domesticated species of bird that is raised for food. So the final answer is chicken.
The relevant information is: Eleanor of Aquitane was born in 1086. So the final answer is 1086 - December 26 , 1086 = 1416.
The important part for answering this question is: VendTransOpen and Vend Trans are different types of transport. So the final answer is VendTrafficOpen and

In [135]:
from sentence_transformers import SentenceTransformer, util
comp_model = SentenceTransformer('all-MiniLM-L6-V2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [136]:
def cosine_similarity_score(pred, target):
    emb1 = comp_model.encode(pred, convert_to_tensor = True)
    emb2 = comp_model.encode(target, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2)[0][0])

In [141]:
print('Target', qa[9]['output'])
print()
print('Prediction', answer)

Target A descriptive statistic is a summary statistic that describes the main features of a dataset, such as the mean, median, mode, standard deviation, and range. These statistics can be used to provide a quick overview of the data and identify patterns or outliers. Other descriptive statistics that can be useful when analyzing a dataset include percentiles, skewness, and kurtosis. These statistics can be calculated using various software tools, such as R or Python, or by using built-in functions in spreadsheet software like Excel.

Prediction In statistics, a descriptive statistic is a statistic that identifies the number or frequency of occurrence of a particular attribute of underlying data. So the final answer is descriptive statistic.


In [142]:
cosine_similarity_score(answer, qa[9]['output'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8961865901947021

In [144]:
cs = []
for i in range(50):
    input = tokenizer(zero_shot_CoT(qa[i]['input']), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cs.append(c)

The relevant information is: Between 1900 and 1920, most of the migrants to the United States came from Europe. So, the final answer is Europe.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The relevant information is: inioluwa means  (inio) olule,  ole, oye, ume. So the final answer is ini ohuwwa.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Oak trees grow at a faster rate during the spring. Spring is the hottest season. So the final answer is during the summer.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [145]:
tem = 0
for i in range(len(cs)):
    tem = tem +cs[i]
print(tem/len(cs))

0.6266356827318669


In [146]:
def zero_shot_CoT(text):
    return f"""
    Q: {text['input']}. A: Let's think step by step and explain this question clearly, (include definition if it has).
    You can recorrect with other reliable sources such as wikipedia if you need,.."""

In [147]:
cs = []
for i in range(50):
    input = tokenizer(zero_shot_CoT(qa[i]), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cs.append(c)
    print(c)

The United States became a migrant destination for many immigrants from China and Southeast Asia. Between 1900 and 1920, the Chinese and Southeast Asian migrants made up the majority of the migrants to the United States. So, the final answer is China.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.7269628047943115
Inioluwa is a word used in the Nigerian lexicon. ... Nigerian language has many words of which inio, a verb, is one of the most common, but ... ... () nio na n  w t iwa (). ... Ini is otu (), n'ot (, ), or  (ekwu, ekwe, olula, ihu wá), , or eke ( , 'she'). ....  is an odio of ma tu.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.893317461013794
The fastest growth in oak trees occurs during the spring and early summer, when the weather is warm and the tree is dormant. So the final answer is during the summer.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8423003554344177


In [148]:
tem = 0
for i in range(len(cs)):
    tem = tem +cs[i]
print(tem/len(cs))

0.5984084041416645


Zero-shot + CoT + SC

In [161]:
cosine = 0
for i in range(3, 53):
    prompt = zero_shot_CoT(qa[i]['input'])
    samples = self_consistency(prompt, num_samples=5)
    for sample in samples:
        print('Sample: ', sample)
        print(cosine_similarity_score(sample, qa[i]['output']))
    print('-------------------------------------------------------')

Sample:  The relevant information to answer the above question is: Chicken is a species of wild or domesticated bird in the family Turdidae. So the final answer is chicken.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.6942747831344604
Sample:  Chicken is a species of monocotyledonous animal. Monocotyledonous animals are divided into two groups based on the number of chromosomes in their cells: heterotrophs and eukary


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.5502927303314209
Sample:  In Christianity, God the Father is the source of love. Jesus is the Son of God. So the final answer is Jesus.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.5130283236503601
-------------------------------------------------------


In [162]:
for i in range(3, 53):
    prompt = zero_shot_CoT(qa[i]['input'])
    samples = self_consistency(prompt, num_samples=3)
    for sample in samples:
        print('Sample: ', sample)
        print(cosine_similarity_score(sample, qa[i]['output']))
    print('-------------------------------------------------------')

Sample:  The relevant information to answer the above question is: Chicken is a species of wild or domesticated bird, typically a hen, but occasionally a young duck or goose. So the final answer is chicken.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.7435966730117798
Sample:  The relevant information to answer the above question is: Chicken is a species of domesticated bird in the family Turdidae. Turdidae is a genus of cyprinid or pulmonate


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.4798750877380371
Sample:  The deficiency in God, the Father, is sin. Jesus is the Son of God, the second Person of the Trinity, who was conceived by the Holy Spirit and born of a virgin. So the final answer is Jesus


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.5224473476409912
-------------------------------------------------------


**Few-shots**

In [ ]:
def few_shots_direct(text):
    return f"""
            Answer the following question, using reliable sources such as wikipedia,...
            Q: "between 1900 and 1920 where did most of the migrants to the united states come from",
            A: "During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world."
                    
            Q: "What does inioluwa mean",
            A: "Inioluwa is a Nigerian name that means \"God's gift\" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God."
                    
            Q: "what is CVD",
            A: "CVD stands for Cardiovascular Disease. It is a group of conditions that affect the heart and blood vessels. These conditions include coronary artery disease (which can lead to heart attacks),"
                    
            Q: {text}
            A: Answer the question                 
                """

In [ ]:
for i in range(3,53):
    input = tokenizer(few_shots_direct(qa[i]['input']), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cosine = cosine+c
    print(c)

In [ ]:
print(cosine/50)

Few-shots + CoT

In [ ]:
def few_shots_CoT(text):
        return f"""
        Answer the following question, using reliable sources such as wikipedia,...
        Q: "between 1900 and 1920 where did most of the migrants to the united states come from",
        A: "During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world."

        Q: "What does inioluwa mean",
        A: "Inioluwa is a Nigerian name that means \"God's gift\" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God."

        Q: "what is CVD",
        A: "CVD stands for Cardiovascular Disease. It is a group of conditions that affect the heart and blood vessels. These conditions include coronary artery disease (which can lead to heart attacks),"

        Q: {text}
        A: Let's think step by step and answer the question                 
                        """

In [ ]:
for i in range(3,53):
    input = tokenizer(few_shots_CoT(qa[i]['input']), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cosine = cosine+c
    print(c)

In [ ]:
print(cosine/50)

Few-shots + CoT + SC

In [112]:
def few_shots_CoT_SC(text, num_samples=5, max_len=50, do_print=False):
        prompt = few_shots_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = functions.majority_vote(samples)
        if do_print:
            print('Answers: ', samples)
            print('Self-consistent answer: ', best_answer)
            print('All votes: ', all_votes)
        return best_answer, all_votes


In [163]:
cosine = 0
for i in range(3, 53):
    prompt = few_shots_CoT(qa[i]['input'])
    samples = self_consistency(prompt, num_samples=5)
    for sample in samples:
        print('Sample: ', sample)
        print(cosine_similarity_score(sample, data[i]['output']))
    print('-------------------------------------------------------')

Sample:  Answer: neutral


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8741482496261597
Sample:  Answer: neutral


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.3955942988395691
Sample:  Answer: neutral


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8741482496261597
-------------------------------------------------------


Lấy thư viện task như 1 tập train lớn và test task sẽ là vùng qa còn lại

In [ ]:
# cosine = 0
# def few_shots_CoT_ART(query):
#     examples = find_top_k_tasks(query, 3)
#     prompt = build_prompt(examples, query)
#     return prompt
# for i in range(1000, 1050):
#     input = tokenizer(few_shots_CoT_ART(qa[i]['input']), return_tensors='pt').to('cuda')
#     output = model.generate(
#         **input,
#     max_length = 300,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     no_repeat_ngram_size=3,
#     early_stopping=False
#     )
#     answer = tokenizer.decode(output[0], skip_special_tokens=True)
#     print(answer)
#     c = cosine_similarity_score(answer, qa[i]['output'])
#     cosine = cosine+c
#     print(c)

Few-shots + ToT

In [110]:
# def few_shots_ToT(text):
#     return f"""
#    Your task: Write 3 possible reasoning paths or situations that might lead to different conclusions and response. Do not answer the question
#     {text['input']} 
#     """
# def followup_prompt(text):
#  return f"""Here are 3 different reasoning paths:  {text}.
#  You are a reasoning assistant. Your job is to explore different reasoning paths without concluding.
#  Now, expand on each path with clear explanation, but do NOT provide any final answer or conclusion."""


In [111]:
# for i in range(50):
#     prompt_base = few_shots_ToT(qa[i])
#     strategies = [generate_strategy(prompt_base, j) for j in range(3)]
#     prompt = prompt_base + "\n" + "\n".join(strategies) + "\n" + "Answer:"
#     input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
#     output = model.generate(
#             **input_ids,
#             max_length=200,
#             num_beams=5,
#             no_repeat_ngram_size=2,
#             early_stopping=True
#         )
#     print(qa[i]['input'])
#     answer = tokenizer.decode(output[0], skip_special_tokens=True)
#     print(answer)

# Computation

**Zero-shot**

Zero-shot + CoT

In [ ]:
import json
math = []
with open('/kaggle/input/final-train-math/final_train_math.json', 'r', encoding='utf-8') as f:
    math.append(json.load(f))

In [ ]:
math = math[0]

In [ ]:
prompt = f"Q: {math[0]['input'] } A: Let's think step by step."
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, max_length=160)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
math[5]['output']

In [ ]:
prompt = f"Q: {math[5]['input'] } A: Let's think step by step. The answer is ?"
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(
    **inputs,
    max_length=100,
    num_beams=2,
    no_repeat_ngram_size=3,
    early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import re

def extract_answer(text):
    matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
    if matches:
        return matches[-1]  
    return None

In [ ]:
answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(answer)

In [ ]:
def extract_target(text):
  for line in text.splitlines()[::-1]:
    if line.strip().startswith("####"):
      return line.strip().replace("####","").strip()
  return None

In [ ]:
acc = 0
results = []
for i in range(20):
  zero_shot_prompt = f"""
  You are a math assistant. When you need to calculate something, use the CALC() function.
  Q: {math[i]['input'] } A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )

  answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1


In [ ]:
print(acc/20)

In [ ]:
acc = 0
results = []
few_shots_prompt = f"""
 Q: {math[0]['input']}. Let's think step by step',
  """
inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=2,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

answer =tokenizer.decode(outputs[0], skip_special_tokens=True)
results.append(answer)

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
print(answer)
print(extract_target(math[1]['output']))

In [ ]:
acc = 0
results = []
for i in range(20):
  zero_shot_prompt = f"""
  You are a math assistant. When you need to calculate something, use the CALC() function.
  Q: {math[i]['input'] } A: You need to answer follow this form: The answer: """
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=200,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(de)
  answer = extract_answer(de)
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1

In [ ]:
print(acc/20)

In [ ]:
for i in range(3):
  print(math[i]['input'])
  print(math[i]['output'])

In [ ]:
clear_gpu()

Zero-shot + CoT + SC

In [ ]:
def self_consistency(prompt, num_samples=5, max_len=150):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=max_len,
            do_sample=True,           # sampling ON
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        print(decoded)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    counts = Counter(answers)
    return counts.most_common(1)[0][0], counts

In [ ]:
def zero_shot_CoT_SC(text):
    return f"""
    f"Q: {text['input'] } A: Let's think step by step. The answer is ?"
    """

In [ ]:
from collections import Counter
prompt = zero_shot_CoT_SC(math[0])
samples = self_consistency(prompt, num_samples=5)
best_answer, all_votes = majority_vote(samples)

print("Self-Consistent Answer:", best_answer)
print("All votes:", all_votes)

In [ ]:
acc = 0
for i in range(50):
    prompt = zero_shot_CoT_SC(math[i])
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    
    print("Self-Consistent Answer:", best_answer)
    print("All votes:", all_votes)
    if best_answer == extract_target(math[i]['output']):
        acc = acc+1

In [ ]:
print(acc/50)

Zero-shot + ToT

In [ ]:
def followup_prompt(text):
 return f"""Here are 3 different reasoning paths:  {text}.
 You are a math assistant. Your job is to explore different reasoning paths without concluding.
 Now, choose the best suitable path with clear explanation and then provide the final answer."""
def zero_shot_ToT(text):
    return f"""
    Q: {text}. A: Answer the question with inferencing
    """
for i in range(50):
    prompt_base = zero_shot_ToT(math[i]['input'])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    prompt = prompt_base + "\n" + "\n".join(strategies) + "\n" + "Answer:"
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
            **input_ids,
            max_length=200,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    print('Target: ', extract_target(math[i]['output']))
    print('Inference: ', math[i]['output'] )
    
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print('Answer: ', extract_answer(answer))
    print('Inference: ', answer )
    

Kết quả rất tệ nếu không hề có 1 ví dụ nào cho mô hình học

**Few-shots**

Few-shots + CoT

In [ ]:
acc  = 0
results = []
for i in range(3,20):
  few_shots_prompt = f"""
  You are a math assistant. Whenever you see a math expression, call the function CALC().
Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
A: Natalia sold CALC(48/2)=24 clips in May.
Natalia sold CALC(48+24)=72 clips altogether in April and May.
#### 72
Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
A: Weng earns $CALC(12/60)=0.2 per minute.
Working 50 minutes, she earned $CALC(0.2*50)=10,
#### 10
Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
A: In the beginning, Betty has only $CALC(100/2) = 50.
Betty's grandparents gave her $CALC(15*2)=30.
This means, Betty needs $CALC(100-50-30-15) = 5 more.
#### 5
Q: {math[i]['input'] } A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

  answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1


In [ ]:
print(acc/17)

In [ ]:
acc  = 0
results = []
for i in range(20):
  few_shots_prompt = f"""Q: Janet’s ducks lay 16 eggs per day.
  She eats three for breakfast every morning and bakes muffins for her friends every day with four.
  She sells the remainder at the farmers' market daily for $2 per fresh duck egg.
  How much in dollars does she make every day at the farmers' market?
 A: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
 The answer: 18.

 Q: {math[i]['input']}. Let's think step by step',
  """
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

  answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1


In [ ]:
print(acc/20)

In [ ]:
import re

def split_steps(text):
    text = text.replace('\n', ' ')
    math_expr = re.findall(r'[\$\d\s\+\-\*/xX=\.]+', text)

    steps = []
    for expr in math_expr:
        expr = expr.strip()
        expr = expr.replace('x', '*').replace('X', '*')  # chuẩn hóa phép nhân
        if '=' in expr and any(c.isdigit() for c in expr):
            steps.append(expr)

    return steps

In [ ]:
def split_cot_and_answer(output_text):
    lines = output_text.strip().splitlines()
    cot_lines = [line for line in lines if not line.strip().startswith("####")]
    answer = extract_target(output_text)
    return "\n".join(cot_lines), answer

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
task_samples = math
new_tasks = []
for task in task_samples:
    cot, answer = split_cot_and_answer(task["output"])
    task["chain_of_thought"] = cot
    task["answer"] = answer
    new_tasks.append(task)
texts = [task['input'] for task in new_tasks]
vectorizer = TfidfVectorizer().fit(texts)
X = vectorizer.transform(texts)

In [ ]:
def find_similar_task(query):
    q = vectorizer.transform([query])
    idx = cosine_similarity(q, X).argmax()
    return task_samples[idx]

one-shot + CoT + ART

In [ ]:
def build_prompt(example, query):
    cot = "\n".join(example["chain_of_thought"])
    return f"""Here is a similar task:
            Task: {example['instruction']}
            Input: {example['input']}
            Chain of Thought:
            {cot}
            Output: {example['output']}
            
            Now solve this task:
            Input: {query}
            Chain of Thought:"""

In [ ]:
def one_shot_ART(query):
    example = find_similar_task(query)
    prompt = build_prompt(example, query)
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(
      **inputs,
      max_length=150,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True) 
    print(answer)
    return answer

In [ ]:
test_tasks= []
with open("/kaggle/input/math-inference/final_test_math (1).json") as f:
    test_tasks = [json.loads(line) for line in f]
for i in range(10):
    one_shot_ART(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

In [ ]:
def build_prompt(example, query):
    cot = "\n".join(example["chain_of_thought"])
    return f"""Here is a similar task:
            Task: {example['instruction']}
            Input: {example['input']}
            Chain of Thought:
            {cot}
            Output: {example['output']}
            Now solve this task:
            Input: {query}
            Solution:
Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"""

In [ ]:
acc = 0
for i in range(20):
    answer = one_shot_ART(test_tasks[i]['input'])
    print('Answer: ', extract_answer(answer))
    target = extract_target(test_tasks[i]['output'])
    print('Target: ', target)
    if extract_answer(answer) == target:
        acc = acc+1

In [ ]:
print(acc/20)

Few-shots + CoT + ART

In [ ]:
def build_prompt(examples, query):
    prompt = ""
    for ex in examples:
        cot = "\n".join(ex.get("chain_of_thought", [])) if "chain_of_thought" in ex else ex["output"]
        prompt += f"""Task: {ex.get('instruction', 'Math Problem')}
                    Input: {ex['input']}
                    Chain of Thought:
                    {cot}
                    Output: {ex['output']}
                    
                    """

    prompt += (
        "Now solve this task:\n"
        f"Input: {query}\n"
        "Solution:\n"
        "Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"
    )
    return prompt

In [ ]:
def find_top_k_tasks(query, k=3):
    q = vectorizer.transform([query])
    scores = cosine_similarity(q, X).flatten()
    top_indices = scores.argsort()[::-1][:k]
    return [task_samples[i] for i in top_indices]

In [ ]:
def few_shots_CoT_ART(query):
    examples = find_top_k_tasks(query, 3)
    prompt = build_prompt(examples, query)
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(
      **inputs,
      max_length=150,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True) 
    return answer

In [ ]:
acc = 0
for i in range(20):
    answer = few_shots_CoT_ART(test_tasks[i]['input'])
    print('Inference', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', extract_target(test_tasks[i]['output']))

In [ ]:
clear_gpu()

In [ ]:
for i in range(10):
    few_shots_ART(test_tasks[i]['input'], 5)
    print(extract_target(test_tasks[i]['output']))

Làm sao để giải quyết được OOM: prompt dài ??? -> prompt compression

Few-shots + CoT(ReAct) + SELF-CONSITENCY (SC) + ART

In [ ]:
def self_consistency(prompt, num_samples=5):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=150,
            do_sample=True,           # sampling ON
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        print(decoded)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    answers = [extract_answer(out) for out in outputs]
    counts = Counter(answers)
    return counts.most_common(1)[0][0], counts

In [ ]:
def build_prompt(examples, query):
    prompt = ""
    for ex in examples:
        cot = "\n".join(ex.get("chain_of_thought", [])) if "chain_of_thought" in ex else ex["output"]
        prompt += f"""Task: {ex.get('instruction', 'Math Problem')}
                    Input: {ex['input']}
                    Chain of Thought:
                    {cot}
                    Output: {ex['output']}
                    
                    """

    prompt += (
        "Now solve this task:\n"
        f"Input: {query}\n"
        "Solution:\n"
        "Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"
    )
    return prompt


In [ ]:
def few_shots_CoT_SC_ART(query):
    examples = find_top_k_tasks(query, 3)
    prompt = build_prompt(examples, query)
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    
    print("Self-Consistent Answer:", best_answer)
    print("All votes:", all_votes)

In [ ]:
for i in range(20):
    few_shots_CoT_SC(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

Few-shots + ToT

In [ ]:
def few_shots_ToT(text):
    return f"""
   Your task: Write 1 possible reasoning path or situation.
    {text} 
    """
def followup_prompt(text):
 return f"""Here are 3 different reasoning paths:  {text}.
 You are a math assistant. Your job is to explore different reasoning paths without concluding.
 Now, expand on each path with clear explanation. The final answer is ?"""
for i in range(20):
    prompt_base = few_shots_ToT(test_tasks[i]['input'])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    prompt = prompt_base + "\n" + "\n".join(strategies) + "\n" + "Answer:"
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
            **input_ids,
            max_length=200,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    print('Target', extract_target(test_tasks[i]['output']))
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print('Inference', answer)
    print('Answer', extract_answer(answer))

# Reasoning

In [ ]:
import json
data= []
file_path = "/kaggle/input/new-infer/reasoning_entries.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

**Zero-shot**

Zero-shot + CoT

In [ ]:
def zero_shot_CoT(text):
    return f"Q: {text['input']}. A: Let's think step by step and explain"

In [ ]:
input = tokenizer(zero_shot_CoT(data[0]), return_tensors="pt").to('cuda')
output = model.generate(
    **input, 
    max_length=100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
clear_gpu()

In [ ]:
cosine = 0
for i in range(50):
    input = tokenizer(zero_shot_CoT(data[i]), return_tensors="pt").to('cuda')
    output = model.generate(
        **input, 
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print('Answer: ', answer)
    print('Target: ', data[i]['output'])
    c = cosine_similarity_score(answer, data[i]['output'])
    print(c)
    cosine = cosine + c

In [ ]:
print(cosine/50)

In [ ]:
def zero_shot_ToT(text):
    return f"Q: {text['input']}. A: explain the main idea and how it could be implemented."
def followup_prompt(text):
 return f"Here are 3 possible situations: {text}. A: Choose the best and explain"


Zero-shot + ToT

In [ ]:
prompt_base = zero_shot_ToT(data[0])
strategies = [generate_strategy(prompt_base, j) for j in range(3)]
print(strategies)
# prompt = prompt_base + "\n" +"\n".join(strategies) + "\n" + "Answer: "
input = tokenizer(followup_prompt(strategies), return_tensors="pt").to('cuda')
output = model.generate(
        **input,
        max_length=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1
    )
answer =tokenizer.decode(output[0], skip_special_tokens=True)
print(answer)

Zero-shot + CoT + SC

**Few-shots**

Few-shots + CoT

In [ ]:
def few_shots_CoT(text):
    return f"""
Q: Explain quantum computing in simple terms,
A: Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. In contrast to classical computers, which use bits to store and process information, quantum computers use quantum bits, or qubits. Because qubits can be in multiple states at once, quantum computers are able to perform many calculations simultaneously, which can make them much faster than classical computers for certain types of problems. While quantum computers are still in the early stages of development, they have the potential to revolutionize many fields, including cryptography, medicine, and materials science.

Q: why shouldn't I trust birds?,
A: There is no reason not to trust birds in general. However, wild birds may carry diseases and it's best to avoid handling them or coming into contact with their droppings. Also, some bird species, such as pigeons, can carry diseases such as histoplasmosis and cryptococcosis. So it's better to take precautions when interacting with wild birds.

Q: I have an array of DomElements in JS, how can I find the item with the largest height?
A: You can loop through the array of DomElements, and for each element, use the `offsetHeight` property to get its height. You can then compare this height to the current maximum height found so far and update the maximum height and corresponding DomElement accordingly. Here's an example implementation:
```javascript
function findMaxHeightElement(elements) {{
  let maxHeight = -Infinity;
  let maxHeightElement = null;

  for (let i = 0; i < elements.length; i++) {{
    const elementHeight = elements[i].offsetHeight;
    if (elementHeight > maxHeight) {{
      maxHeight = elementHeight;
      maxHeightElement = elements[i];
    }}
  }}

  return maxHeightElement;
}}
Q: {text['input']}
A: Let's think step by step
"""

In [ ]:
cosine = 0
for i in range(20):
    input = tokenizer(few_shots_CoT(data[i]), return_tensors="pt").to('cuda')
    output = model.generate(
        **input, 
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    target = data[i]['output']
    print(answer)
    c = cosine_similarity_score(answer, target)
    print(c)
    cosine = cosine + c

In [ ]:
print(cosine/20)

Few-shots + CoT + SC

Few-shots ToT

In [ ]:
def few_shots_ToT(text):
    return f"""
Q: Explain quantum computing in simple terms,
A: Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. In contrast to classical computers, which use bits to store and process information, quantum computers use quantum bits, or qubits. Because qubits can be in multiple states at once, quantum computers are able to perform many calculations simultaneously, which can make them much faster than classical computers for certain types of problems. While quantum computers are still in the early stages of development, they have the potential to revolutionize many fields, including cryptography, medicine, and materials science.

Q: why shouldn't I trust birds?,
A: There is no reason not to trust birds in general. However, wild birds may carry diseases and it's best to avoid handling them or coming into contact with their droppings. Also, some bird species, such as pigeons, can carry diseases such as histoplasmosis and cryptococcosis. So it's better to take precautions when interacting with wild birds.

Q: I have an array of DomElements in JS, how can I find the item with the largest height?
A: You can loop through the array of DomElements, and for each element, use the `offsetHeight` property to get its height. You can then compare this height to the current maximum height found so far and update the maximum height and corresponding DomElement accordingly. Here's an example implementation:
```javascript
function findMaxHeightElement(elements) {{
  let maxHeight = -Infinity;
  let maxHeightElement = null;

  for (let i = 0; i < elements.length; i++) {{
    const elementHeight = elements[i].offsetHeight;
    if (elementHeight > maxHeight) {{
      maxHeight = elementHeight;
      maxHeightElement = elements[i];
    }}
  }}

  return maxHeightElement;
}}
Q: {text['input']}
A:  A:Let's think of three different strategies for this question. For each strategy, explain the main idea and how it could be implemented.
"""

In [ ]:
cosine = 0
for i in range(20):
    prompt_base = few_shots_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    # prompt = prompt_base + "\n" +"\n".join(strategies) + "\n" + "Answer: "
    input = tokenizer(followup_prompt(strategies), return_tensors="pt").to('cuda')
    output = model.generate(
            **input,
            max_length=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            num_return_sequences=1
        )
    answer =tokenizer.decode(output[0], skip_special_tokens=True)
    target = data[i]['output']
    print(answer)
    c = cosine_similarity_score(answer, target)
    print(c)
    cosine = cosine + c

In [ ]:
print(cosine/20)

# Finetune

In [ ]:
import json
classification = []
with open('classification_instruction_data_en.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        classification.append(json.loads(line.strip()))
qa = []
computation = []
reasoning = []

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
os.environ["WANDB_DISABLED"] = "true"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model2 = T5ForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False
)
trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model2)
)
trainer.train()